In [1]:
from atproto import Client
from datetime import datetime

C:\Users\Christina Benopoulou\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'default' attribute with value None was provided to the `Field()` function, which has no effect in the context it was used. 'default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [15]:
BLUESKY_HANDLE = "socialweb-project.bsky.social"      # e.g. "christina123.bsky.social"
BLUESKY_APP_PASSWORD = "qfdm-t2h6-dhbm-4l2b"   # the one you just created

In [16]:
ELECTION_YEAR = 2025

In [17]:
female_politicians = [
    "Fatihya Abdi", "Elles van Ark", "Etkin Armut", "Bente Becker",
    "Sandra Beckerman", "Fatimazhra Belhirch", "Nathalie van Berkel",
    "Anouschka Biekman", "Mirjam Bikker", "Lucienne Boelsma-Hoekstra",
    "Corrie van Brenk", "Laura Bromet", "Judith Buhler",
    "Ingrid Coenradie", "Heera Dijk", "Inge van Dijk",
    "Sarah Dobbe", "Wendy van Eijk", "Sarah El Boujdaini",
    "Renate den Hollander", "Tamara ten Hove", "Renilde Huizenga",
    "Barbara Kathmann", "Mona Keijzer", "Ines Kostic",
    "Suzanne Kroger", "Esmah Lahlah", "Annelotte Lammers",
    "Vicky Maeijer", "Claire Martens-America", "Rachel van Meetelen",
    "Ingrid Michon-Derkzen", "Nicole Moinat", "Marjolein Moorman",
    "Songul Mutluer", "Annabel Nanninga", "Sjoukje van Oosterhout",
    "Ouafa Oualhadj", "Esther Ouwehand", "Mariette Patijn",
    "Wieke Paulusma", "Kati Piri", "Caroline van der Plas",
    "Anne-Marijke Podt", "Annette Raijer", "Queeny Rajkowski",
    "Ilana Rooderkerk", "Shanna Schilder", "Hanneke Steen",
    "Jeltje Straatman", "Marijke Synhaeve", "Christine Teunissen",
    "Eveline Tijmstra", "Mikal Tseggai",
    "Marieke Vellinga-Beemsterboer", "Lisa Vliegenthart",
    "Marina Vondeling", "Lidewij de Vos", "Hilde Wendel",
    "Hanneke van der Werf", "Lisa Westerveld", "Femke Wiersma",
    "Dilan Yesilgoz-Zegerius", "Ani Zalinyan", "Jantine Zwinkels"
]

print(f"Configuration loaded. Searching for {len(female_politicians)} female politicians in {ELECTION_YEAR}.")

Configuration loaded. Searching for 65 female politicians in 2025.


In [18]:
client = Client()
client.login(BLUESKY_HANDLE, BLUESKY_APP_PASSWORD)
print(f"Connected to Bluesky as: {client.me.handle}")

Connected to Bluesky as: socialweb-project.bsky.social


In [19]:
bluesky_posts = []

for politician in female_politicians:
    print(f"Searching Bluesky for: {politician}")
    
    # First page
    resp = client.app.bsky.feed.search_posts(params={"q": politician, "limit": 50})
    posts = resp.posts
    
    for post in posts:
        record = post.record
        content = getattr(record, "text", "") or ""
        created_at = getattr(record, "created_at", None)
        
        if created_at:
            timestamp = datetime.fromisoformat(created_at.replace("Z", "+00:00"))
            
            if timestamp.year == ELECTION_YEAR and politician.lower() in content.lower():
                rkey = post.uri.split("/")[-1]
                url = f"https://bsky.app/profile/{post.author.handle}/post/{rkey}"
                
                bluesky_posts.append({
                    "platform": "Bluesky",
                    "text": content,
                    "author": post.author.handle,
                    "created_at": timestamp,
                    "url": url,
                    "politician": politician
                })
    
    # Pagination
    cursor = resp.cursor
    while cursor:
        resp = client.app.bsky.feed.search_posts(
            params={"q": politician, "limit": 50, "cursor": cursor}
        )
        posts = resp.posts
        
        if not posts:
            break
        
        for post in posts:
            record = post.record
            content = getattr(record, "text", "") or ""
            created_at = getattr(record, "created_at", None)
            
            if created_at:
                timestamp = datetime.fromisoformat(created_at.replace("Z", "+00:00"))
                
                if timestamp.year < ELECTION_YEAR:
                    break
                
                if timestamp.year == ELECTION_YEAR and politician.lower() in content.lower():
                    rkey = post.uri.split("/")[-1]
                    url = f"https://bsky.app/profile/{post.author.handle}/post/{rkey}"
                    
                    bluesky_posts.append({
                        "platform": "Bluesky",
                        "text": content,
                        "author": post.author.handle,
                        "created_at": timestamp,
                        "url": url,
                        "politician": politician
                    })
        
        cursor = resp.cursor

print(f"\nCollected {len(bluesky_posts)} Bluesky posts")

Searching Bluesky for: Fatihya Abdi
Searching Bluesky for: Elles van Ark
Searching Bluesky for: Etkin Armut
Searching Bluesky for: Bente Becker
Searching Bluesky for: Sandra Beckerman
Searching Bluesky for: Fatimazhra Belhirch
Searching Bluesky for: Nathalie van Berkel
Searching Bluesky for: Anouschka Biekman
Searching Bluesky for: Mirjam Bikker
Searching Bluesky for: Lucienne Boelsma-Hoekstra
Searching Bluesky for: Corrie van Brenk
Searching Bluesky for: Laura Bromet
Searching Bluesky for: Judith Buhler
Searching Bluesky for: Ingrid Coenradie
Searching Bluesky for: Heera Dijk
Searching Bluesky for: Inge van Dijk
Searching Bluesky for: Sarah Dobbe
Searching Bluesky for: Wendy van Eijk
Searching Bluesky for: Sarah El Boujdaini
Searching Bluesky for: Renate den Hollander
Searching Bluesky for: Tamara ten Hove
Searching Bluesky for: Renilde Huizenga
Searching Bluesky for: Barbara Kathmann
Searching Bluesky for: Mona Keijzer
Searching Bluesky for: Ines Kostic
Searching Bluesky for: Suzanne

In [20]:
all_posts = bluesky_posts


print("="*70)
print("COLLECTION SUMMARY")
print("="*70)
print(f"Total posts collected: {len(all_posts)}")
print(f"  - Bluesky: {len(bluesky_posts)}")
print(f"Number of female politicians searched: {len(female_politicians)}")


print("\nSample posts (first 5):")
for i, post in enumerate(all_posts[:5], 1):
    print(f"\n{i}. [{post['platform']}] About {post['politician']}")
    print(f"   By: {post['author']}")
    print(f"   Date: {post['created_at']}")
    print(f"   Text: {post['text'][:100]}...")
    print(f"   URL: {post['url']}")

COLLECTION SUMMARY
Total posts collected: 15884
  - Bluesky: 15884
Number of female politicians searched: 65

Sample posts (first 5):

1. [Bluesky] About Fatihya Abdi
   By: stemvizier.nl
   Date: 2025-11-28 09:05:36.475596+00:00
   Text: De motie van Fatihya Abdi (#GLPvdA) c.s. over overleggen over de impact van de voorgenomen bezuinigi...
   URL: https://bsky.app/profile/stemvizier.nl/post/3m6ojs5h7xm2v

2. [Bluesky] About Fatihya Abdi
   By: emmacurvers.bsky.social
   Date: 2025-11-22 16:50:00.175000+00:00
   Text: Haha oké. Er zijn altijd ‘weggestemden’, toch? Hoezo onterecht dan? Deze man had weinig voorkeursste...
   URL: https://bsky.app/profile/emmacurvers.bsky.social/post/3m6aawch4gs2y

3. [Bluesky] About Fatihya Abdi
   By: johnsocialnow.bsky.social
   Date: 2025-11-13 01:43:54.203000+00:00
   Text: Hoe kijkt u (Fatihya Abdi) daar tegen aan? @fatihyaabdi.bsky.social...
   URL: https://bsky.app/profile/johnsocialnow.bsky.social/post/3m5i23rukvs2g

4. [Bluesky] About Fatihya Ab

In [25]:

hate_keywords = {
    'sexist': [
        # English
        'bitch', 'slut', 'whore', 'feminazi', 'should be home', 'kitchen', 
        'belong in kitchen', 'stay home', 'not fit for', 'too emotional',
        'hormonal', 'on her period', 'typical woman', 'like all women',
        'gold digger', 'sleeping her way', 'token woman', 'diversity hire',
        # Dutch
        'hoer', 'slet', 'teef', 'wijf', 'kutwijf', 'trut', 'tuthola',
        'blijf thuis', 'thuishoren', 'keuken', 'niet geschikt',
        'typisch vrouw', 'als alle vrouwen', 'emancipatiehoer'
    ],
    
    'appearance': [
        # English
        'ugly', 'fat', 'old', 'haggard', 'looks like', 'hideous', 'disgusting',
        'needs makeup', 'aging', 'wrinkled', 'unattractive', 'gross', 'obese',
        'pig', 'cow', 'dog', 'horse face', 'too old', 'past her prime',
        'should dress better', 'unfuckable', 'butter face',
        # Dutch
        'lelijk', 'dik', 'oud', 'afgetakeld', 'ziet eruit als', 'goor',
        'walgelijk', 'smerig', 'varken', 'koe', 'hond', 'te oud', 
        'ouwe taart', 'opgespoten', 'plastisch'
    ],
    
    'threats': [
        # English
        'kill', 'rape', 'assault', 'death threat', 'should die', 'hope you die',
        'deserve to die', 'hang', 'shoot', 'stab', 'murder', 'assassinate',
        'violence', 'hurt', 'beat', 'attack', 'lynch', 'burn',
        'going to get', 'coming for you', 'watch your back', 'better watch out',
        # Dutch
        'dood', 'verkracht', 'vermoord', 'doodgeschoten', 'opgehangen',
        'gestoken', 'aanval', 'geweld', 'pijn doen', 'slaan', 'pak op je',
        'krijg je', 'komt ervan', 'kijk maar uit', 'pas maar op'
    ],
    
    'misogyny': [
        # English
        'stupid woman', 'dumb female', 'emotional', 'hysterical', 'irrational',
        'cant think', 'no brain', 'empty head', 'airhead', 'bimbo', 'ditzy',
        'woman logic', 'female brain', 'overly sensitive', 'dramatic',
        'attention seeker', 'attention whore', 'playing victim', 'victim card',
        'women are', 'females are', 'all women', 'typical female',
        # Dutch
        'domme vrouw', 'stomme wijf', 'emotioneel', 'hysterisch', 'irrationeel',
        'geen hersens', 'leeg hoofd', 'dom blondje', 'vrouwenlogica',
        'typisch vrouwelijk', 'aandachtshoer', 'slachtoffer spelen',
        'vrouwen zijn', 'alle vrouwen', 'wijven zijn'
    ],
    
    'objectification': [
        # English
        'hot', 'sexy', 'body', 'breasts', 'legs', 'ass', 'boobs', 'tits',
        'would bang', 'would smash', 'milf', 'piece of ass', 'nice rack',
        'fuckable', 'bangable', 'doable', 'eye candy', 'arm candy',
        'prostitute', 'escort', 'stripper', 'pornstar',
        # Dutch
        'lekker', 'geil', 'lichaam', 'borsten', 'tieten', 'kont', 'benen',
        'neuken', 'pikken', 'pijpen', 'hoer', 'prostituee', 'slet',
        'hoerig', 'sletterig', 'goedkoop'
    ],
    
    'competence_undermining': [
        # English
        'unqualified', 'incompetent', 'clueless', 'has no idea', 'doesnt know',
        'out of her depth', 'in over her head', 'puppet', 'controlled by',
        'token', 'quota', 'affirmative action', 'only there because',
        'diversity', 'check box', 'not earned', 'handed to her',
        # Dutch
        'onbekwaam', 'incompetent', 'geen idee', 'weet niet', 'snapt niet',
        'marionet', 'pop', 'aangestuurd door', 'quotum', 'alleen maar omdat',
        'diversiteit', 'niet verdiend', 'cadeau gekregen'
    ],
    
    'dismissive': [
        # English
        'shut up', 'nobody cares', 'who cares', 'go away', 'disappear',
        'resign', 'step down', 'quit', 'get out', 'fuck off', 'piss off',
        'go back to', 'return to', 'not welcome', 'dont belong',
        # Dutch
        'kop dicht', 'hou je bek', 'bek houden', 'niemand interesseert',
        'boeit niet', 'rot op', 'oprotten', 'wegwezen', 'ga weg',
        'opzouten', 'opdonderen', 'aftreden', 'opstappen', 'ga terug naar',
        'niet welkom', 'hoort hier niet'
    ],
    
    'ethnic_religious_slurs': [
        # English (relevant to Dutch political context)
        'muslim', 'immigrant', 'foreign', 'terrorist', 'go back', 'not dutch',
        'invasion', 'replacement', 'traitor', 'islam lover', 'multiculti',
        # Dutch
        'moslim', 'allochtoon', 'buitenlander', 'terrorist', 'ga terug',
        'niet nederlands', 'invasie', 'vervanging', 'landverraad', 'verrader',
        'islamliefhebber', 'gutmensch', 'linkse rat'
    ]
}
for post in all_posts:
    text_lower = post['text'].lower()
    post['hate_speech'] = False
    post['hate_categories'] = []
    
    for category, keywords in hate_keywords.items():
        if any(keyword in text_lower for keyword in keywords):
            post['hate_speech'] = True
            post['hate_categories'].append(category)


total = len(all_posts)
hate_count = len([p for p in all_posts if p['hate_speech']])

print(f"Analysis Complete:")
print(f"Total posts: {total}")
print(f"Posts with potential hate speech: {hate_count} ({hate_count/total*100:.1f}%)")
print(f"\nBreakdown by category:")

for category in hate_keywords.keys():
    count = len([p for p in all_posts if category in p.get('hate_categories', [])])
    print(f"  - {category}: {count}")


Analysis Complete:
Total posts: 15884
Posts with potential hate speech: 3850 (24.2%)

Breakdown by category:
  - sexist: 233
  - appearance: 2124
  - threats: 677
  - misogyny: 20
  - objectification: 705
  - competence_undermining: 359
  - dismissive: 48
  - ethnic_religious_slurs: 155
